In [1]:
## Importing the required libraries
import numpy as np
import tensorflow as tf
from tensorflow import keras

After importing the libraries, we will specify the value for the hyperparameters including the batch size for training, 
latent dimensionality for the encoding space and the number of samples to train on.

In [2]:
## Hyperparameters
batch_size = 64  # Batch size for training.
epochs = 10  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "fra.txt"

The below lines of codes will perform the data vectorization where we will read the file containing the English and 
corresponding French sentences. In the vectorization process, the collection of text documents is converted into feature vectors.

## Preparing the data

In [3]:
# Vectorize the data.
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 92
Max sequence length for inputs: 15
Max sequence length for outputs: 59


## Build the Model

In [4]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

## Train the model

In [5]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s1")

Epoch 1/10
125/125 [==============================] - 57s 392ms/step - loss: 1.1525 - accuracy: 0.7346 - val_loss: 1.0508 - val_accuracy: 0.7187
Epoch 2/10
125/125 [==============================] - 50s 395ms/step - loss: 0.8102 - accuracy: 0.7794 - val_loss: 0.8559 - val_accuracy: 0.7649
Epoch 3/10
125/125 [==============================] - 49s 393ms/step - loss: 0.6411 - accuracy: 0.8191 - val_loss: 0.6812 - val_accuracy: 0.8067
Epoch 4/10
125/125 [==============================] - 48s 387ms/step - loss: 0.5623 - accuracy: 0.8369 - val_loss: 0.6288 - val_accuracy: 0.8145
Epoch 5/10
125/125 [==============================] - 48s 386ms/step - loss: 0.5166 - accuracy: 0.8486 - val_loss: 0.5854 - val_accuracy: 0.8292
Epoch 6/10
125/125 [==============================] - 47s 380ms/step - loss: 0.4827 - accuracy: 0.8578 - val_loss: 0.5540 - val_accuracy: 0.8381
Epoch 7/10
125/125 [==============================] - 53s 426ms/step - loss: 0.4552 - accuracy: 0.8649 - val_loss: 0.5306 - val_ac

INFO:tensorflow:Assets written to: s2s1\assets


INFO:tensorflow:Assets written to: s2s1\assets


## Run Iference
- encode input and retrieve initial decoder state
- run one step of decoder with this initial state and a "start of sequence" token as target. Output will be the next target token.
- Repeat with the current target token and current state

In [6]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s1")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence


In [8]:
for seq_index in range(30):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Déchamez-le.

-
Input sentence: Go.
Decoded sentence: Déchamez-le.

-
Input sentence: Go.
Decoded sentence: Déchamez-le.

-
Input sentence: Hi.
Decoded sentence: Ai comment !

-
Input sentence: Hi.
Decoded sentence: Ai comment !

-
Input sentence: Run!
Decoded sentence: Autez-le.

-
Input sentence: Run!
Decoded sentence: Autez-le.

-
Input sentence: Run!
Decoded sentence: Autez-le.

-
Input sentence: Run!
Decoded sentence: Autez-le.

-
Input sentence: Run!
Decoded sentence: Autez-le.

-
Input sentence: Run!
Decoded sentence: Autez-le.

-
Input sentence: Run!
Decoded sentence: Autez-le.

-
Input sentence: Run!
Decoded sentence: Autez-le.

-
Input sentence: Run.
Decoded sentence: Fais !

-
Input sentence: Run.
Decoded sentence: Fais !

-
Input sentence: Run.
Decoded sentence: Fais !

-
Input sentence: Run.
Decoded sentence: Fais !

-
Input sentence: Run.
Decoded sentence: Fais !

-
Input sentence: Run.
Decoded sentence: Fais !

-
Input sentence: Ru